### Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial import distance
import random
import math

### Load 2D GT data  & its corresponding 3D data
Noise is only added to 2D data as the 2D detector can not be precise at all in some situations and this may be a good approach to improve generalization

In [2]:
columns_df_2D_interest = ['right_shoulder_2D','right_shoulder_2D.1',
                          'right_elbow_2D','right_elbow_2D.1',
                          'right_wrist_2D', 'right_wrist_2D.1',          
                          'left_shoulder_2D','left_shoulder_2D.1',  
                          'left_elbow_2D','left_elbow_2D.1',
                          'left_wrist_2D','left_wrist_2D.1']

columns_df_3D_interest = ['right_shoulder','right_shoulder.1','right_shoulder.2',
                          'right_elbow', 'right_elbow.1','right_elbow.2',
                          'right_wrist','right_wrist.1','right_wrist.2',
                          'left_shoulder','left_shoulder.1','left_shoulder.2',
                          'left_elbow', 'left_elbow.1', 'left_elbow.2',
                          'left_wrist','left_wrist.1','left_wrist.2']

In [3]:
#2D data
df_p1_2D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion1_Raycast_50k.csv')[columns_df_2D_interest]
df_p2_2D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion2_Raycast_50k.csv')[columns_df_2D_interest]
df_p3_2D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion3_Raycast_50k.csv')[columns_df_2D_interest]
df_p4_2D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion4_Raycast_50k.csv')[columns_df_2D_interest]
df_p5_2D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion5_Raycast_50k.csv')[columns_df_2D_interest]
df_p6_2D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion6_Raycast_50k.csv')[columns_df_2D_interest]


df_2D_original = df_p1_2D_original.append(df_p2_2D_original).reset_index().drop(columns=['index'])
df_2D_original = df_2D_original.append(df_p3_2D_original).reset_index().drop(columns=['index'])
df_2D_original = df_2D_original.append(df_p4_2D_original).reset_index().drop(columns=['index'])
df_2D_original = df_2D_original.append(df_p5_2D_original).reset_index().drop(columns=['index'])
df_2D_original = df_2D_original.append(df_p6_2D_original).reset_index().drop(columns=['index'])

print(df_2D_original.shape)
df_2D_original.head(5)

(300000, 12)


,right_shoulder_2D,right_shoulder_2D.1,right_elbow_2D,right_elbow_2D.1,right_wrist_2D,right_wrist_2D.1,left_shoulder_2D,left_shoulder_2D.1,left_elbow_2D,left_elbow_2D.1,left_wrist_2D,left_wrist_2D.1
0,810.276,393.482,546.963,359.344,316.919,503.028,1122.844,391.740,1410.094,279.290,1791.517,198.782
1,753.873,393.354,550.222,423.395,398.479,592.313,1037.152,403.974,1321.294,371.415,1655.104,336.437
2,764.024,394.902,469.788,344.342,-3.661,269.685,1024.258,404.961,1241.366,499.301,1441.945,591.105
3,765.325,388.454,446.753,268.330,38.184,116.024,1075.513,395.992,1333.211,357.750,1600.414,427.706
4,802.265,383.466,482.957,307.194,43.690,199.115,1127.084,381.821,1423.897,369.169,1502.276,568.855


In [4]:
# 3D data

df_p1_3D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion1_Raycast_50k.csv')[columns_df_3D_interest]
df_p2_3D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion2_Raycast_50k.csv')[columns_df_3D_interest]
df_p3_3D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion3_Raycast_50k.csv')[columns_df_3D_interest]
df_p4_3D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion4_Raycast_50k.csv')[columns_df_3D_interest]
df_p5_3D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion5_Raycast_50k.csv')[columns_df_3D_interest]
df_p6_3D_original = pd.read_csv('../datasets/train/004_train_dataset/Seccion6_Raycast_50k.csv')[columns_df_3D_interest]


df_3D_original = df_p1_3D_original.append(df_p2_3D_original).reset_index().drop(columns=['index'])
df_3D_original = df_3D_original.append(df_p3_3D_original).reset_index().drop(columns=['index'])
df_3D_original = df_3D_original.append(df_p4_3D_original).reset_index().drop(columns=['index'])
df_3D_original = df_3D_original.append(df_p5_3D_original).reset_index().drop(columns=['index'])
df_3D_original = df_3D_original.append(df_p6_3D_original).reset_index().drop(columns=['index'])

print(df_3D_original.shape)
df_3D_original.head(5)

(300000, 18)


,right_shoulder,right_shoulder.1,right_shoulder.2,right_elbow,right_elbow.1,right_elbow.2,right_wrist,right_wrist.1,right_wrist.2,left_shoulder,left_shoulder.1,left_shoulder.2,left_elbow,left_elbow.1,left_elbow.2,left_wrist,left_wrist.1,left_wrist.2
0,-0.212,-0.207,1.357,-0.605,-0.265,1.404,-0.849,-0.049,1.266,0.227,-0.206,1.334,0.588,-0.340,1.251,0.934,-0.383,1.076
1,-0.282,-0.201,1.313,-0.638,-0.182,1.493,-0.825,0.077,1.408,0.116,-0.205,1.441,0.493,-0.230,1.308,0.863,-0.253,1.190
2,-0.267,-0.198,1.308,-0.533,-0.213,1.042,-0.804,-0.226,0.799,0.096,-0.202,1.433,0.423,-0.061,1.439,0.738,0.078,1.468
3,-0.270,-0.210,1.330,-0.626,-0.331,1.169,-0.974,-0.448,1.013,0.170,-0.212,1.413,0.569,-0.278,1.461,0.917,-0.161,1.372
4,-0.222,-0.220,1.348,-0.573,-0.280,1.151,-0.908,-0.338,0.950,0.231,-0.219,1.327,0.645,-0.238,1.333,0.683,0.036,1.206


### Apply gaussian noise lvls to each 2D keypoint

- Low level --> 0-15 px
- Medium level --> 0-30 px
- High level --> 0-50 px

Change the circle_r variable in order to apply one lvl of noise or another. 

### Apply Noise to datapoints in order to increase generalization 

The applied noise will vary from 0 to a specific radius circumference. 
The value of the radius will determine the level of noise

In [5]:
def split_2_coordinates(values_array): 
    output = []
    for i in range(0, len(values_array)):
        m = 0
        r = 2
        frame_coordinates = []
        for j in range(0,int(len(values_array[i])/2)):
            frame_coordinates.append(values_array[i][m:r])
            m = r
            r = r+2
        output.append(frame_coordinates)
    return(output)

In [6]:
df_2D_original_splitted = pd.DataFrame(split_2_coordinates(np.array(df_2D_original)))
df_2D_original_splitted.head(5)

,0,1,2,3,4,5
0,"[810.2760000000001, 393.48199999999997]","[546.9630000000001, 359.344]","[316.91900000000004, 503.028]","[1122.844, 391.74]","[1410.094, 279.29]","[1791.517, 198.782]"
1,"[753.873, 393.35400000000004]","[550.222, 423.395]","[398.47900000000004, 592.313]","[1037.152, 403.974]","[1321.2939999999999, 371.415]","[1655.104, 336.43699999999995]"
2,"[764.0239999999999, 394.902]","[469.788, 344.342]","[-3.661, 269.685]","[1024.258, 404.961]","[1241.366, 499.301]","[1441.945, 591.105]"
3,"[765.325, 388.454]","[446.75300000000004, 268.33]","[38.184, 116.024]","[1075.513, 395.99199999999996]","[1333.211, 357.75]","[1600.414, 427.70599999999996]"
4,"[802.265, 383.466]","[482.957, 307.194]","[43.69, 199.115]","[1127.0839999999998, 381.82099999999997]","[1423.8970000000002, 369.16900000000004]","[1502.276, 568.855]"


In [7]:
df_2D_noise1 = []
for i in range(0,df_2D_original_splitted.shape[0]-1):
    row_keypoint_numbers = []
    for j in range(0,df_2D_original_splitted.shape[1]):
        
        circle_r = 30 #Maximum radius
        
        circle_x = df_2D_original_splitted.iloc[i][j][0]
        circle_y = df_2D_original_splitted.iloc[i][j][1]
        
        alpha = 2 * math.pi * random.random()
        # random radius
        r = circle_r * math.sqrt(random.random())
        
        # calculating coordinates
        x = r * math.cos(alpha) + circle_x
        x = round(x,2)
        
        y = r * math.sin(alpha) + circle_y
        y = round(y,2)
        
        row_keypoint_numbers.append(x)
        row_keypoint_numbers.append(y)
    df_2D_noise1.append(row_keypoint_numbers)
    
    
df_2D_noise1 = pd.DataFrame(df_2D_noise1)
    

       

In [8]:
df_2D_noise1

,0,1,2,3,4,5,6,7,8,9,10,11
0,833.58,377.76,534.83,346.27,329.65,487.35,1093.22,389.37,1415.39,294.57,1806.40,182.38
1,748.79,394.60,532.04,415.93,388.26,564.55,1024.74,399.72,1324.07,351.29,1671.60,360.19
2,736.12,404.78,442.18,352.32,8.70,291.91,1033.64,429.04,1222.72,489.52,1448.49,610.83
3,772.86,384.62,434.38,292.70,41.62,107.51,1055.16,406.39,1321.42,334.24,1616.38,422.38
4,798.17,412.68,479.08,283.99,52.62,175.23,1115.58,408.55,1410.90,346.31,1484.87,584.03
...,...,...,...,...,...,...,...,...,...,...,...,...
299994,801.63,409.67,706.60,187.17,475.95,127.66,1094.81,384.98,1072.49,180.73,749.96,300.21
299995,801.14,374.20,649.57,143.93,580.33,31.11,1099.09,407.91,921.17,294.82,608.22,275.03
299996,751.56,405.03,646.86,235.26,485.36,331.90,1068.97,363.11,1087.56,156.49,829.50,303.52
299997,845.16,367.74,674.11,244.09,587.33,388.68,1133.01,381.69,886.73,472.36,699.39,682.80


In [9]:
df_2D_noise1_splitted.to_csv('../datasets/train/004_train_dataset/NoiseLVL2_2D_Raycast_300k.csv')

NameError: name 'df_2D_noise1_splitted' is not defined

### Checking if gaussian noise is being applied correctly

Seleccionar 200 datos 2D original y 200 datos de cada uno de los datasets generados a distinito nivel gausiano y calcular la distáncia euclideana de los 6 keypoints entre ellos

In [13]:
def split_2_coordinates(values_array): 
    output = []
    for i in range(0, len(values_array)):
        m = 0
        r = 2
        frame_coordinates = []
        for j in range(0,int(len(values_array[i])/2)):
            frame_coordinates.append(values_array[i][m:r])
            m = r
            r = r+2
        output.append(frame_coordinates)
    return(output)

In [14]:
df_2D_noise1_splitted = pd.DataFrame(split_2_coordinates(np.array(df_2D_noise1_splitted)))

In [17]:
all_distances = [] #2D
for i in range(0,df_2D_noise1_splitted.shape[1]):
    keypoint_distances = []
    for j in range(0,len(df_2D_original_splitted[i])):
        try:
            keypoint_distances.append(distance.euclidean(df_2D_original_splitted[i][j], df_2D_noise1_splitted[i][j]))
        except:
            pass
    all_distances.append(keypoint_distances)

In [18]:
np.array(all_distances).mean()

10.000349047605347